In [1]:
from typing import Annotated
from langgraph.prebuilt.tool_node import InjectedState
from langchain_core.tools import tool
from app.firebase.firestore import (
    add_event,
    get_events,
    update_event_by_id,
    delete_event_by_id
)

@tool
def create_event(user_id: Annotated[str, InjectedState("user_id")], event_description: str):
    """Create a new surveillance event to monitor."""
    return add_event(user_id, {"description": event_description})

@tool
def read_events(user_id: Annotated[str, InjectedState("user_id")]):
    """Read all active surveillance events"""
    return get_events(user_id)

@tool
def update_event(user_id: Annotated[str, InjectedState("user_id")], event_id: str, updated_description: str):
    """Update a specific surveillance event with a new description."""
    return update_event_by_id(user_id, event_id, {"description": updated_description})

@tool
def delete_event(user_id: Annotated[str, InjectedState("user_id")], event_id: str):
    """Delete an active surveillance event."""
    return delete_event_by_id(user_id, event_id)

tools = [create_event, read_events, update_event, delete_event]


In [2]:
import sqlite3
from langgraph.graph import MessagesState, StateGraph, START, END
from langchain_core.messages import HumanMessage, SystemMessage, RemoveMessage
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import ToolNode, tools_condition
from typing import List, Optional
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
db_path = "chatbot_memory.db"
conn = sqlite3.connect(db_path, check_same_thread=False)
memory = SqliteSaver(conn)

In [4]:
llm = ChatOpenAI(model="gpt-4o", temperature=0)
llm_with_tools = llm.bind_tools(tools)

In [5]:
class ChatbotState(MessagesState):
    summary: Optional[str] = None
    user_id: str

In [6]:
# --- Assistant Node ---
system_message = SystemMessage(
    content="You are a home surveillance assistant. Help users manage security events. Use tools to perform actions like create, update, delete, or read events."
)

def assistant_node(state: ChatbotState):
    messages = state["messages"]
    summary = state.get("summary")
    if summary:
        context = SystemMessage(content=f"Summary of earlier conversation: {summary}")
        messages = [context] + messages
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

In [8]:
# --- Summarize Node ---
def summarize_conversation(state: ChatbotState):
    summary = state.get("summary", "")
    recent = state["messages"][-2:]
    summary_prompt = (
        f"This is the current summary: {summary}\n\n"
        "Update the summary based on the new conversation above."
        if summary else
        "Summarize the conversation above:"
    )
    messages = state["messages"] + [HumanMessage(content=summary_prompt)]
    response = llm.invoke(messages)
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]

    return {
        "summary": response.content,
        "messages": recent + delete_messages,
    }

In [9]:
# --- Condition: Decide whether to end or summarize ---
def should_continue(state: ChatbotState):
    if len(state["messages"]) > 6:
        return "summarize_conversation"
    return END

In [10]:
# --- LangGraph Construction ---
builder = StateGraph(ChatbotState)
builder.add_node("assistant", assistant_node)
builder.add_node("tools", ToolNode(tools))
builder.add_node("summarize_conversation", summarize_conversation)

builder.set_entry_point("assistant")

# Assistant node -> tool or END
builder.add_conditional_edges("assistant", tools_condition)
builder.add_edge("tools", "assistant")
builder.add_conditional_edges("assistant", should_continue)
builder.add_edge("summarize_conversation", END)

# Compile with memory
chat_graph = builder.compile(checkpointer=memory)

In [11]:
# --- Public async function to use chatbot ---
async def run_chatbot_agent(message: str, user_id: str) -> str:
    input_msg = HumanMessage(content=message)
    config = {"configurable": {"thread_id": user_id}}

    result = await chat_graph.ainvoke({"messages": [input_msg],"user_id" : user_id}, config )
    messages: List = result["messages"]

    return messages[-1].content if messages else "No response generated."


In [12]:
response = await run_chatbot_agent(message="Hi", user_id="8xokvbpU0WUxMraKmWkriTtzjym2")

NotImplementedError: The SqliteSaver does not support async methods. Consider using AsyncSqliteSaver instead.
from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver
Note: AsyncSqliteSaver requires the aiosqlite package to use.
Install with:
`pip install aiosqlite`
See https://langchain-ai.github.io/langgraph/reference/checkpoints/#langgraph.checkpoint.sqlite.aio.AsyncSqliteSaverfor more information.

In [ ]:
for chuck in response:
    print(chuck,end="\n")